In [11]:
with open("hf.token", "r") as f:
    hftoken = f.read().strip()  

import os
cache_dir = "/mnt/c/Users/yc/.cache/huggingface"
os.environ['HF_HOME'] = cache_dir

from huggingface_hub import login
login(token=hftoken)  # Move token to environment variable

from ult import *

## load model

In [12]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:09<00:00, 17.36s/it]


## data

In [13]:
def txt_to_dict(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines) - 1, 2):
            key = lines[i].strip()    # Odd line are key
            value = lines[i + 1].strip()  # Even line are value
            data_dict[key] = value

    return data_dict

txt_file_path = 'data/formaldef.txt'
formaldic = txt_to_dict(txt_file_path)
len(formaldic)

meddict={}
for k,v in formaldic.items():
    meddict[k.split('Listen to pronunciation')[0].split('(')[0]]=v
filename= 'data/filtered_medical_dictionary.csv'
eighth_grade_words=set()
with open(filename, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        if row:  # Make sure row is not empty
            eighth_grade_words.add(row[0])  # Add the word (first column)
filtered_meddict = {word: explanation for word, explanation in meddict.items() 
                   if word in eighth_grade_words}
meddict=filtered_meddict
# load data
df = pd.read_csv('data/CORAL/coral-expert-curated-medical-oncology-reports-to-advance-language-model-inference-1.0/coral/unannotated/data/breastca_unannotated.csv')
df = df.sample(1, random_state=42) 
test_note=df.iloc[0]['note_text']
len(df)

1

# working

In [14]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Dict, List, Optional, Tuple

def run_model(prompt_text: str, model, tokenizer, generation_config: Dict) -> str:
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    input_length = inputs["input_ids"].shape[1]

    if "pad_token_id" not in generation_config:
        generation_config["pad_token_id"] = tokenizer.eos_token_id

    with torch.no_grad():
        attention_mask = inputs.get("attention_mask", torch.ones_like(inputs["input_ids"]))

        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=attention_mask,
            **generation_config
        )

    response_tokens = outputs[0][input_length:]
    raw_output = tokenizer.decode(response_tokens, skip_special_tokens=True)
    return raw_output.strip()

def find_term_in_dict(term: str, meddict: dict) -> Optional[str]:
    if term in meddict:
        return meddict[term]
    
    term_lower = term.lower()
    for key in meddict:
        if key.lower() == term_lower:
            return meddict[key]
    
    words = term.split()
    if len(words) > 1:
        results = []
        for word in words:
            result = find_term_in_dict(word, meddict)
            if result:
                results.append(result)
        if results:
            return " ".join(results)
    
    return ""

def parse_lookup_request(text: str) -> Optional[str]:
    match = re.search(r'\[LOOKUP:\s*(.*?)\s*\]', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

def explain_interactively(
    original_text: str,
    model,
    tokenizer,
    meddict: Dict[str, str],
    base_generation_config: Dict,
    max_lookups: int = 5
) -> Tuple[str, List[Dict]]:
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    if "eos_token_id" not in base_generation_config or base_generation_config["eos_token_id"] != terminators:
         base_generation_config["eos_token_id"] = terminators
    if "pad_token_id" not in base_generation_config:
         base_generation_config["pad_token_id"] = tokenizer.eos_token_id

    lookup_count = 0
    conversation_history = []

    system_prompt = """### Persona
You are an experienced and compassionate Oncologist writing a simple explanation for a patient (8th-grade level). Your goal is radical simplicity.

### Your Task
Explain the key information from the provided medical text directly to the patient (using 'you'). Structure your explanation clearly.

### CRITICAL TOOL USAGE:
If you encounter a specific medical term (like a diagnosis, procedure, or complex drug name) that you believe needs a simple definition for the patient to understand, you MUST output the special tag `[LOOKUP: The Term Here]`. output the special tag [LOOKUP: The Term Here] for any specific medical diagnosis, procedure name, genetic mutation, or drug name mentioned, even if you think you know it. Do NOT use it for common words like 'lung' or 'scan'. After you output the tag, STOP generating immediately. You will be provided with the definition, and then you must continue your explanation, incorporating that definition simply. Do NOT include the `[LOOKUP: ...]` tag in your final explanation to the patient."""

    user_prompt = f"""Please explain the following medical information to me in simple terms:

"{original_text}"
"""

    conversation_history.append({"role": "system", "content": system_prompt})
    conversation_history.append({"role": "user", "content": user_prompt})

    current_explanation = ""

    while lookup_count < max_lookups:
        prompt_text = tokenizer.apply_chat_template(
            conversation_history,
            tokenize=False,
            add_generation_prompt=True
        )

        generated_chunk = run_model(prompt_text, model, tokenizer, base_generation_config)
        generated_chunk = generated_chunk.strip()

        term_to_lookup = parse_lookup_request(generated_chunk)

        if term_to_lookup:
            text_before_lookup = generated_chunk.split(f"[LOOKUP:{term_to_lookup}]", 1)[0].strip()
            if text_before_lookup:
                 current_explanation += text_before_lookup + " "
                 conversation_history.append({"role": "assistant", "content": text_before_lookup})

            lookup_count += 1

            definition = find_term_in_dict(term_to_lookup, meddict)
            print(f'!! look up res {definition}')
            if not definition:
                definition = f"No simple definition found for '{term_to_lookup}' in the dictionary."

            tool_response = f"Definition requested for '{term_to_lookup}': {definition}. Please continue your explanation simply, incorporating this definition naturally."
            conversation_history.append({"role": "user", "content": tool_response})

            continue
        else:
            current_explanation += generated_chunk + " "
            conversation_history.append({"role": "assistant", "content": generated_chunk})
            break

    final_explanation = current_explanation.strip()
    final_explanation = re.sub(r'\[LOOKUP:\s*(.*?)\s*\]', '', final_explanation).strip()
    final_explanation = re.sub(r'\s+', ' ', final_explanation)

    return final_explanation, conversation_history
if __name__ == "__main__":

    # Use a config similar to your EXPLANATION_GENERATION_CONFIG, but ensure EOS is handled
    agent_gen_config = {
        "max_new_tokens": 999, # Generate smaller chunks in the loop
        "temperature": 0.3,
        "top_p": 0.85,
        "repetition_penalty": 1.1,
        "do_sample": True,
        "early_stopping": True, # Stop if EOS is generated
        # eos/pad tokens handled within explain_interactively
    }
    test_note2=', '.join(list(meddict.keys())[100:110])
    test_note2 = "The patient's chart mentions an ACTHoma and also actinic keratosis. The proposed treatment is actinomycin D, following the results of the action study."
    for i in range(100,110):
        print(list(meddict.keys())[i])
        print(list(meddict.values())[i])
        print('\n')
    final_explanation, history = explain_interactively(
        test_note2,
        model,
        tokenizer,
        meddict,
        agent_gen_config,
        max_lookups=5
    )

    print("\n--- Final Explanation ---")
    print(final_explanation)

    print("\n--- Full Conversation History ---")
    for i, msg in enumerate(history):
        print(f"{i+1}. Role: {msg['role']}")
        print(f"   Content: {msg['content']}") # Print snippet
        print(msg)


ACTHoma
A rare tumor that makes the hormone adrenocorticotropin (ACTH), which causes the adrenal glands to make too much cortisol. ACTHomas can lead to Cushing syndrome. Signs and symptoms of ACTHomas and Cushing syndrome include a round face, thin arms and legs, severe fatigue and muscle weakness, high blood pressure, high blood sugar, purple or pink stretch marks on the skin, and weight gain, especially in the abdomen. ACTHomas usually form in the pituitary gland or in the islet cells of the pancreas, but they may also form in other organs, including the lung, thyroid, and thymus. ACTHomas may be benign (not cancer) or malignant (cancer). Also called adrenocorticotropic-secreting tumor.


Actinex
A drug put on the skin to treat growths caused by sun exposure. A form of Actinex that is taken by mouth is being studied in the treatment of prostate cancer. Actinex is an antioxidant, and it may block certain enzymes needed for tumor growth. Also called masoprocol, NDGA, and nordihydroguai

# prompt

In [ ]:

        
def clean_model_output(text: str, fix_incomplete=True) -> str:
    """
    Clean up model-generated text with common fixes, not currently using
    """
    if not text:
        return ""
    
    # Basic cleanup
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)  # Normalize all whitespace
    
    # Fix paragraph spacing
    text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
    
    # Fix punctuation spacing
    text = re.sub(r'\s+([,.!?;:])', r'\1', text)
    text = re.sub(r'([.!?])\s*([A-Z])', r'\1 \2', text)
    
    # Remove repetitive patterns (simple version)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if not cleaned_lines or line.strip() != cleaned_lines[-1].strip():
            cleaned_lines.append(line)
    text = '\n'.join(cleaned_lines)
    
    # Handle incomplete sentences
    if fix_incomplete and text and not text.endswith(('.', '!', '?')):
        sentences = re.split(r'[.!?]+', text)
        if len(sentences) > 1 and len(sentences[-1].strip()) < 10:
            # Remove likely incomplete last sentence
            text = '.'.join(sentences[:-1]) + '.'
    
    return text.strip()

def extract_medical_terms(text: str, meddict: Dict[str, str]) -> Dict[str, str]:

    found_terms = {}
    
    # Strategy 1: Single words
    words = re.findall(r'\b[A-Za-z]+(?:[-\'][A-Za-z]+)*\b', text)
    for word in words:
        definition = find_term_in_dict(word, meddict)
        if definition:
            found_terms[word] = definition
    
    # Strategy 2: Multi-word terms
    for n in range(2, 6):
        n_grams = get_n_grams(text, n)
        for phrase in n_grams:
            definition = find_term_in_dict(phrase, meddict)
            if definition:
                found_terms[phrase] = definition
    
    # Strategy 3: Medical abbreviations
    abbreviations = re.findall(r'\b[A-Z]{2,8}\b', text)
    for abbrev in abbreviations:
        definition = find_term_in_dict(abbrev, meddict)
        if definition:
            found_terms[abbrev] = definition
    
    # Strategy 4: Medical procedures and conditions with specific patterns
    medical_patterns = [
        r'\b\w+oscopy\b',          # bronchoscopy, endoscopy, etc.
        r'\b\w+ectomy\b',          # appendectomy, etc.
        r'\b\w+itis\b',            # bronchitis, arthritis, etc.
        r'\b\w+osis\b',            # fibrosis, stenosis, etc.
        r'\b\w+emia\b',            # anemia, septicemia, etc.
        r'\b\w+pathy\b',           # myopathy, neuropathy, etc.
        r'\b\w+malacia\b',         # tracheomalacia, etc.
    ]
    
    for pattern in medical_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            definition = find_term_in_dict(match, meddict)
            if definition:
                found_terms[match] = definition
    
    # Strategy 5: Medication names
    medication_patterns = [
        r'\b\w+cillin\b',          # penicillin, amoxicillin, etc.
        r'\b\w+mycin\b',           # streptomycin, etc.
        r'\b\w+floxacin\b',        # levofloxacin, ciprofloxacin, etc.
        r'\b\w+sone\b',            # prednisone, cortisone, etc.
        r'\b\w+pam\b',             # lorazepam, etc.
    ]
    
    for pattern in medication_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            definition = find_term_in_dict(match, meddict)
            if definition:
                found_terms[match] = definition
    
    return found_terms


def get_n_grams(text: str, n: int) -> List[str]:
    """Generate n-grams from text."""
    words = re.findall(r'\b[A-Za-z]+\b', text.lower())
    n_grams = []
    for i in range(len(words) - n + 1):
        phrase = ' '.join(words[i:i+n])
        n_grams.append(phrase)
    return n_grams


def find_term_in_dict(term: str, meddict: Dict[str, str]) -> Optional[str]:
    """Find term in medical dictionary."""
    search_formats = [
        term, term.lower(), term.upper(), term.title(), term.capitalize()
    ]
    
    for search_term in search_formats:
        if search_term in meddict:
            return meddict[search_term]
    
    # Partial matching
    for key in meddict.keys():
        if key.lower() == term.lower():
            return meddict[key]
    
    return None

def create_annotated_text(text: str, meddict: Dict[str, str]) -> str:

    # 1. Use the existing function to find all unique terms and their definitions.
    found_terms = extract_medical_terms(text, meddict)
    
    # 2. Sort terms by length in descending order to handle overlaps.
    # This is critical for terms like "cerebral palsy" and "palsy".
    sorted_terms = sorted(found_terms.keys(), key=len, reverse=True)
    
    annotated_text = text
    
    # 3. Iterate and replace.
    for term in sorted_terms:
        definition = found_terms[term]
        annotation = f"{term} [DEFINITION: {definition}]"
        pattern = r'\b' + re.escape(term) + r'\b'
        annotated_text = re.sub(pattern, annotation, annotated_text, count=1, flags=re.IGNORECASE)
        
    return annotated_text


def create_key_summary_prompt(original_text: str) -> str:
    messages = [
        {"role": "system", "content": """You are an expert medical information extractor with exceptional attention to detail. Your task is to carefully read the medical text and extract ONLY the key factual details that are explicitly mentioned. You must be extremely precise and never infer, assume, or add any information not directly stated in the text.
CRITICAL RULES:
1. Extract ONLY facts explicitly stated in the text
2. Use the EXACT wording from the original text when possible
3. Do not interpret, infer, or elaborate beyond what is written
4. If multiple items exist in a category, separate them with semicolons
5. Keep each section concise but complete
6. If a category is not mentioned or unclear, write "Not mentioned" for that section."""},
        {"role": "user", "content": f"""**Medical Text:**\n"{original_text}"\n\n**Format your response EXACTLY as follows:**\n\n**TREATMENTS RECEIVED:**\n[List treatments]\n\n**MEDICAL CONDITIONS:**\n[List conditions]\n\n**CANCER STAGE:**\n[List stage]\n\n**REFERRALS:**\n[List referrals]\n\n**CURRENT STATUS:**\n[List status]\n\n**NEXT STEPS/PLAN:**\n[List plan]"""}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


def create_explanation_prompt(annotated_text: str, keysummary: str) -> str:

    audience_instruction = "The determined audience for this letter is the **patient**. You must address them directly as 'you' throughout the entire letter."
    
    system_prompt = f"""### Persona
You are an experienced and compassionate Oncologist and medical educator. Your primary role is to translate complex medical information into clear, 8th-grade level English. Your tone should be professional, empathetic, and honest.

### Golden Rule: Radical Simplicity - Translate, Don't Transfer
Your single most important task is to convert medical terminology into simple explanations. Do not just define a term; replace it with an easy-to-understand explanation.

### Your Task:
Write a complete letter explaining the information from the medical text. Imagine you are sitting with the recipient and explaining this to them in person.

{audience_instruction}

### Letter Structure and Flow:
Organize your letter logically using the following question-based headers:
* **Empathetic Opening:** Start with a warm and supportive salutation.
* **Why you came today:** Briefly explain the purpose of the visit in plain words.
* **What you told us:** Summarize what the patient and family shared (symptoms, worries, questions).
* **What did we find** Explain what the team learned from today's visit (exam, blood work, scans) in simple terms.
* **What is the plan** List the next steps (treatment, monitoring, follow-up) in plain language.
* **Closing with Support:** End by reinforcing your support. Limit to one sentence maximum.

### STRICT NEGATIVE CONSTRAINTS:
* Do not say anything not present in the medical note.
* If cancer has spread (metastasis), DO NOT list the specific organs affected. Say "the cancer has spread to other parts of the body."
* Do not use fatalistic language. The focus MUST be on quality of life.

### Strict Output Formatting:
Provide ONLY the letter. Your output must start directly with "Dear [Patient Name]," and end with "Sincerely,\nYour Care Team at [Institution]". Use the question-based headers exactly as specified."""
    
    user_prompt = f"""### Original Medical Text:
"{annotated_text}"

### Internal Fact-Checking Reference (Translate these facts, do not copy them):
"{keysummary}"

Please provide the patient-focused explanation now. Your output must start directly with the salutation."""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



def create_cleaning_prompt(raw_response: str) -> str:
    """
    Creates the correctly formatted prompt for Llama 3 to clean/revise a response.
    """
    system_prompt = f"""### Persona
You are an expert medical writer and editor. Your unique skill is communicating complex clinical information with absolute precision and clarity. Your primary directive is to preserve the original meaning without fail.

### Target Audience: patient

### Your Task
Revise the provided medical text to improve its quality in the following areas:
1.  **Clean Up Language:** Improve sentence structure and use professional language.
2.  **Reduce Repetition:** Eliminate redundant words and phrases.
3.  **Improve Flow:** Enhance the logical flow and transitions.
4.  **Define Medical Terms** For any complex medical term, replace it with a simple explanation. Example: "you had tachycardia."--> "you had a fast heart rate".
5.  **Remove formatting subtitles:** remove any section headers or subtitles from the text to make it read as a continuous letter.
### The Golden Rule: Preserve Clinical Meaning at All Costs
The revised text MUST be factually identical to the original.
* **DO NOT** alter any clinical facts, diagnoses, measurements, or timelines.
* **DO NOT** change the certainty of a statement.

### Instructions for Output
Provide your Revised Medical Text: The complete, revised version of the text."""
    user_prompt = f"""### Medical Text to Revise:
{raw_response}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    return tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


KEY_SUMMARY_CONFIG = {
    "max_new_tokens": 500,
    "do_sample": False, 
    "repetition_penalty": 1.1,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": terminators, 
}

EXPLANATION_GENERATION_CONFIG = {
    "max_new_tokens": 1024,
    "temperature": 0.3,
    "top_p": 0.85,
    "repetition_penalty": 1.15,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": terminators, 
    "early_stopping": True,
}

CLEANING_CONFIG = {
    "max_new_tokens": 1500,
    "temperature": 0.1,
    "top_p": 0.9,
    "repetition_penalty": 1.1,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": terminators, 
    "early_stopping": True,
}



def run_model(prompt: str, model, tokenizer, generation_config: Dict) -> str:

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
        input_length = inputs["input_ids"].shape[1]
        
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                **generation_config
            )
        response_tokens = outputs[0][input_length:]
        raw_output = tokenizer.decode(response_tokens, skip_special_tokens=True)
        return raw_output
        


all_results = {}
LINE_WIDTH = 100 

for index, row in df.iterrows():
    text = row['note_text'] 
    
    print(f"\nProcessing row {index + 1}/{len(df)}...")
    
    # --- Step 1: Determine Audience ---
    audience = 'patient' 
    
    # --- Step 2: Annotate Text (Non-LLM step) ---
    annotated_text = create_annotated_text(text, meddict)
    
    # --- Step 3: Extract Key Summary ---
    summary_prompt = create_key_summary_prompt(text)
    keysummary = run_model(summary_prompt, model, tokenizer, KEY_SUMMARY_CONFIG)
    
    # --- Step 4: Generate Initial Explanation ---
    explanation_prompt = create_explanation_prompt(annotated_text, keysummary)
    explanation = run_model(explanation_prompt, model, tokenizer, EXPLANATION_GENERATION_CONFIG)

    # --- Step 5: Clean and Finalize the Explanation ---
    cleaning_prompt = create_cleaning_prompt(explanation)
    final_result = run_model(cleaning_prompt, model, tokenizer, CLEANING_CONFIG)
    
    # --- Store and Print Results ---
    row_result = {
        'original_text': text,
        # 'annotated_text': annotated_text,
        'keysummary': keysummary,
        'raw_explanation': explanation,
        'final_letter': final_result
    }
    
    all_results[index]=(row_result)
    print(f"\n{'='*60}")
    print(f"RESULTS FOR ROW {index + 1}")
    print(f"{'='*60}")
    
    for col in ['keysummary', 'raw_explanation', 'final_letter']:
        print(f"\n--- Column: {col} ---")
        original_text = row_result[col]
        wrapped_text = textwrap.fill(original_text, width=LINE_WIDTH)
        print(wrapped_text)


Processing row 84/1...


/home/yc/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yc/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/yc/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



RESULTS FOR ROW 84

--- Column: keysummary ---
**TREATMENTS RECEIVED:** 1. Zolendronic acid;  2. Letrozole and Palbociclib;  3. Capecitabine
(Xeloda);  4. Interferon beta-1a (Avonex)  **MEDICAL CONDITIONS:** 1. Metastatic breast cancer;  2.
Multiple sclerosis;  3. CHEK2 mutation;  4. Liver disease;  5. Possible leptomeningeal disease;  6.
Bone disease;  7. Soft tissue disease;   **CANCER STAGE:** Metastatic  **REFERRALS:** 1. Referral to
radiation oncology;  2. Referral to UCSF Rad Onc for opinion  **CURRENT STATUS:** Progressively
worsening neurological symptoms;  Diffuse irregular enhancement along the bilateral cerebral
convexities and right petroclival region;  Possible worsening of disease in the internal auditory
canal;  Recent right hearing loss;  No improvement in vision;  False negative CSF cytology result;
Extensive lytic bone lesions;  Metastatic disease to the liver and pachymeninges;  Possible
metastatic disease to the leptomeninges;  Progression of disease on Xeloda;  Ex

In [ ]:
# results_df = pd.DataFrame(all_results).T
# # results_df.drop(columns=['annotated_text'], inplace=True)
# output_format = 'csv'

# # --- File Saving Logic ---
# base_filename = 'output/output'
# extension = ''

# if output_format == 'csv':
#     extension = '.csv'
# elif output_format == 'json':
#     extension = '.json'
# else:
#     print(f"Error: Unsupported format '{output_format}'. Please choose 'csv' or 'json'.")
#     exit() # Exits the script if the format is invalid


# output_filename = base_filename + extension
# counter = 1

# output_dir = os.path.dirname(base_filename)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# while os.path.exists(output_filename):
#     output_filename = f"{base_filename}_{counter}{extension}"
#     counter += 1

# if output_format == 'csv':
#     results_df.to_csv(output_filename, index=True)
# elif output_format == 'json':
#     results_df.to_json(output_filename, orient='index', indent=2)

# print(f"DataFrame saved to '{output_filename}'")

DataFrame saved to 'output/output_12.csv'
